In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import cv2
import pathlib
import torch
import torchvision.transforms as transforms
from torchvision.io import read_image

DATA_DIR = "facial_expressions/data"
IMAGE_DIR = "facial_expressions/images"

data = pd.read_csv(f'{DATA_DIR}/legend.csv')
data = data.head(300)
# data = pd.read_csv(f'{DATA_DIR}/500_picts_satz.csv', names=['user', 'image', 'emotion'])
data['emotion'] = data['emotion'].str.lower()
data.rename(columns={'image': 'path'}, inplace=True)
data['image'] = [torch.unsqueeze(torch.Tensor(plt.imread(f'{IMAGE_DIR}/{path}').astype('float32')), 0) for path in data['path']]

In [2]:
data['shape'] = [img.shape for img in data['image']]
data.describe()

,user.id,path,emotion,image,shape
count,300,300,300,300,300
unique,3,300,7,300,4
top,302,facial-expressions_2868588k.jpg,neutral,"[[[tensor([203., 201., 197.]), tensor([203., 2...","(1, 350, 350)"
freq,295,1,182,1,296


In [3]:
same_shape_data = data[data['shape'] == (1, 350, 350)]

,user.id,path,emotion,image,shape
4,dwdii,Aaron_Eckhart_0001.jpg,neutral,"[[[tensor(9.), tensor(9.), tensor(9.), tensor(...","(1, 350, 350)"
5,302,Aaron_Guiel_0001.jpg,happiness,"[[[tensor(10.), tensor(10.), tensor(10.), tens...","(1, 350, 350)"
6,302,Aaron_Patterson_0001.jpg,neutral,"[[[tensor(92.), tensor(92.), tensor(92.), tens...","(1, 350, 350)"
7,302,Aaron_Peirsol_0001.jpg,happiness,"[[[tensor(125.), tensor(125.), tensor(118.), t...","(1, 350, 350)"
8,302,Aaron_Peirsol_0002.jpg,happiness,"[[[tensor(82.), tensor(82.), tensor(83.), tens...","(1, 350, 350)"
...,...,...,...,...,...
295,302,Alex_Cabrera_0001.jpg,neutral,"[[[tensor(47.), tensor(47.), tensor(49.), tens...","(1, 350, 350)"
296,302,Alex_Cejka_0001.jpg,happiness,"[[[tensor(18.), tensor(18.), tensor(17.), tens...","(1, 350, 350)"
297,302,Alex_Corretja_0001.jpg,neutral,"[[[tensor(47.), tensor(47.), tensor(46.), tens...","(1, 350, 350)"
298,302,Alex_Ferguson_0001.jpg,happiness,"[[[tensor(89.), tensor(90.), tensor(91.), tens...","(1, 350, 350)"


In [4]:
# tensor_img = torch.Tensor(data['image'])
# tensor_label = torch.Tensor(data['emotion'])
# convert_tensor = transforms.ToTensor()
# convert_tensors = np.vectorize(convert_tensor)
# ts = [convert_tensor(img) for img in images]
# tensor_big = torch.Tensor(ts)
# torch.unsqueeze(torch.Tensor(images[0]), 0).shape

In [14]:
from torch.utils.data import Dataset
import pandas as pd
import os
from torchvision.io import read_image

class FacialExpression(Dataset):
  def __init__(self, annotations_file=f'{DATA_DIR}/legend.csv', img_dir=IMAGE_DIR, transform=None, target_transform=None):
    self.img_dir = img_dir
    self.data = same_shape_data
    self.classes = list(self.data['emotion'].unique())
    self.data['class'] = [self.classes.index(e) for e in self.data['emotion']]
    self.img_labels = self.data['class']
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    image = self.data['image'][idx]
    label = self.img_labels[idx]
    if self.transform:
        image = self.transform(image)
    if self.target_transform:
        label = self.target_transform(label)
    return image, label

In [15]:
trainset = FacialExpression()

/tmp/ipykernel_14728/3822882487.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['class'] = [self.classes.index(e) for e in self.data['emotion']]


In [16]:
batch_size = 4
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [17]:
# import matplotlib.pyplot as plt
# import numpy as np
# import torchvision

# # functions to show an image


# def imshow(img):
#     # img = img / 2 + 0.5     # unnormalize
#     # npimg = img.numpy()
#     # plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.imshow(img, cmap="gray")
#     plt.show()


# # get some random training images
# dataiter = iter(trainloader)
# images, labels = next(dataiter)

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(' '.join(f'{labels[j]:5s}' for j in range(batch_size)))

In [18]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        # self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(112896, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [19]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [20]:
for epoch in range(2):  # loop over the dataset multiple times

  running_loss = 0.0
  for i, batch in enumerate(trainloader, 0):
    print(i)
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = batch

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
      print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
      running_loss = 0.0

print('Finished Training')

yo 189yo 76

po 76po 189

yo 128yo 83

po 128po 83

yo 93yo 221

po 93po 221

yo 240yo 133

po 240po 133

yo 92yo 116

po 92po 116

yo 245yo 49

po 245po 49

yo 38
po 38
yo 145yo 105

po 145po 105

yo 91
po 91
yo 104
po 104
yo 295
po 295
yo 173
po 173
yo 127
po 127
yo 60
po 60
yo 283
po 283
yo 155
po 155
yo 39
po 39
yo 40
po 40
yo 293
po 293
yo 146
po 146
yo 3
0
1
2
3
yo 73
po 73
yo 86
po 86
yo 135
po 135
yo 291
po 291
yo 26
po 26
yo 87
po 87
yo 267
po 267
yo 153
po 153
yo 234
po 234
yo 124
po 124
yo 107
po 107
yo 159
po 159
yo 81
4
5
po 81

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/tjhu/.local/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3803, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 2263, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 2273, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 3

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/tjhu/.local/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/tjhu/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py"


yo 259
po 259
yo 228
po 228
yo 143


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cpu


In [ ]:
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
p = model(data['image'][0])

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 536, 347, 3] to have 3 channels, but got 536 channels instead